In [84]:
import numpy as np
import pandas as pd
import cv2
import os
import pandas as pd

# import keras.layers as kr

import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('train.csv')
grp = train.groupby(['x','y'])
grp.any()

,,id
x,y,
-171.180678,73.78272,True
-163.780678,78.38272,True
-158.680678,22.68272,True
-158.380678,19.78272,True
-158.180678,81.98272,True
...,...,...
86.719322,-24.31728,True
87.319322,-26.71728,True
88.619322,-28.81728,True


In [38]:
def load_images(fol):
    images = []
    for filename in os.listdir(fol):
        img = cv2.imread(os.path.join(fol,filename),cv2.IMREAD_GRAYSCALE)
        if img is not None:
            images.append((filename,img))
    return images

In [39]:
train_images = load_images('train')
test_images = load_images('test')

In [40]:
print(len(test_images))
print(len(train_images))

1200
7500


In [43]:
train_img = train_images[0]
test_img = test_images[0]

In [49]:
# SIFT feature matching 
# Initiate SIFT detector
def sift(train_img,test_img):
    sift = cv2.SIFT_create() # if cv2 version >= 4.4.0 
    # sift = cv2.xfeatures2d.SIFT_create() # if cv2 version = 4.3.x 

    # Compute SIFT keypoints and descriptors
    kp1, des1 = sift.detectAndCompute(train_img,None)
    kp2, des2 = sift.detectAndCompute(test_img,None)

    # FLANN parameters and initialize
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)   # or pass empty dictionary
    flann = cv2.FlannBasedMatcher(index_params,search_params)

    # Matching descriptor using KNN algorithm
    matches = flann.knnMatch(des1,des2,k=2)

    # Store all good matches as per Lowe's Ratio test.
    ratio = 0.7
    good = []
    for m,n in matches:
        if m.distance < ratio*n.distance:
            good.append(m)
    
    return len(good)
    # Set a condition that at least MIN_MATCH_NUM matches are required to find the object. 
#     MIN_MATCH_NUM = 4
    

#     if len(good)> MIN_MATCH_NUM:
#         return True
#         # If enough matches are found, we extract the positions of the matched keypoints in both images. 
#         # They are passed to find the perspective transformation. 

#         # Estimate homography between two images
# #         ptsA = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1,1,2)
# #         ptsB = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1,1,2)
# #         H, status = cv2.findHomography(ptsA, 
# #                                        ptsB, 
# #                                        cv2.RANSAC, 
# #                                        ransacReprojThreshold = 5, 
# #                                        maxIters = 500)

# #         matchesMask = status.ravel().tolist()

#         # Draw detected template in scene image
# #         imgOut = cv2.warpPerspective(test_img, H, (train_img.shape[1],train_img.shape[0]),
# #                                  flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)

#         # Print total number of successful matches between two images
# #         print("\nNumber of successful matches between two images: ", matchesMask.count(1)) # Returns the number of 1 in the success list

#     else:
#         return False
#         # Otherwise, print that “Not enough matches are found”.
#         print( "Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_NUM) )
#         matchesMask = None


#     # Draw our inliers (if successfully found the object) or all matching keypoints (if failed)
#     draw_params = dict(matchColor = (0,255,0), # draw in a specific colour
#                        singlePointColor = (255,0,0), # draw in a specific colour
#                        matchesMask = matchesMask, # draw only inliers
#                        flags = 2)

#     success_matches = cv2.drawMatches(train_img,kp1,test_img,kp2,good,None,**draw_params)


#     # Plotting results
#     plt.subplots(figsize=(10, 10)) 

#     if matchesMask == None:
#         plt.imshow(success_matches)
#         plt.title('All matching keypoints')
#         plt.axis('off')

#     else:
#         plt.subplot(2,1,1)
#         plt.imshow(success_matches)
#         plt.title('All successful matches')
#         plt.axis('off')

#         plt.subplot(2,1,2)
#         plt.imshow(imgOut, 'gray')
#         plt.title('Display detected template in scene image')
#         plt.axis('off')

#     plt.show()

In [65]:
test_name,test_img = test_images[0]
matched = []

for train_name,train_img in train_images[::5]:
    matched.append((test_name,train_name,sift(train_img,test_img)))
            
        

In [87]:
test_image,best_match,_ = max(matched,key=lambda x:x[2])
coor = train[train['id']== best_match[:-4]]
output = {'id': test_image[:-4], 'x': coor['x'] ,'y': coor['y']}  
output = pd.DataFrame(output,columns= ['id', 'x','y'])  

output.to_csv (r'test.csv', index = False, header=True)